In [ ]:
pip install qdrant-client sentence_transformers

In [3]:
import pandas as pd

df = pd.read_csv('/kaggle/input/sai-literature/modified.csv')
df.columns
# df = df[['Unnamed: 0','book','topic','content']]

Index(['book', 'topic', 'content'], dtype='object')

In [4]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models

client = QdrantClient(url="https://ccaa5582-c81c-49c3-93a9-866c4617ce0e.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="S1RryvNzKIXBwLYriv2FVqj11oUgoyrrxyLJDNfiyi5he7b6SRs-jg",)
modelst = SentenceTransformer("msmarco-distilbert-dot-v5")

# client.create_collection(
#     collection_name="database",
#     hnsw_config=models.HnswConfigDiff(
#         m=81,
#         ef_construct=200,
#     ),
#     vectors_config=models.VectorParams(
#         size=768,
#         distance=models.Distance.COSINE,
#     ),
# )

In [ ]:
client.upload_points(
    collection_name="database",
    points=[
        models.PointStruct(
            id=i,
            vector=modelst.encode(str(obj['content'])).tolist(),
            payload=dict(obj)
        )
        for i,obj in temp.iterrows()
    ],
)

In [ ]:
client.count(
    collection_name='database'
)

In [7]:
def user_search(qn):
    
    hits = client.search(
        collection_name="database",
        query_vector=modelst.encode(qn).tolist(),
        limit=10,
        search_params=models.SearchParams(hnsw_ef=128,exact=True),
    )

    res = []
    
    print('\nTop 5 results!!\n')
    
    for hit in hits:
        
        if hit.score > 77:
            res.append(hit.payload)
            
        print(hit.payload)
        print('score = ',hit.score,'\n')
        
    return res

In [ ]:
# from sentence_transformers.cross_encoder import CrossEncoder
# import numpy as np

# ranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-12-v2")

# def ranking(query,paras):
#     texts = [(query,para['content']) for para in paras]
#     scores = ranker.predict(texts)

#     # Sort the scores in decreasing order to get the corpus indices
#     ranked_indices = np.argsort(scores)[::-1]

#     print('after re-ordering\n')
#     for r in ranked_indices:
#         print(paras[r],'\n score = ',scores[r],'\n')

In [ ]:
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
from transformers import BitsAndBytesConfig
import torch

config = BitsAndBytesConfig(
    load_in_4bit=True, #changing the model weights to bit floating points
    bnb_4bit_compute_dtype=torch.float16, #the computational matrices that will be used for training or inference in floating point 16
    bnb_4bit_quant_type="nf4", #this is the quantization type
    bnb_4bit_use_double_quant=True, #doing double quantization
)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained("/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1",quantization_config=config,device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1")

In [ ]:
messages = []

In [ ]:
def generate(context,query):
    
#     print('entered for generation')
    global messages 
        
    prompt = f"""
    Chat History is given below.
    ---------------------
    {messages}
    ---------------------
    Context information is below.
    ---------------------
    Context:
    {context}
    ---------------------
    Given the context and chat history, answer the query.
    Query: {query}
    Instructions:
        - Generate a comprehensive response by leveraging the context effectively to address the query cohesively, ensuring the information provided is relevant and coherent within the given context. 
        - Leverage the usage of transliterations,romanizations, anecdotes or short stories from the context to answer the query.    
        - When the context is "None," utilize the chat history as a reference to guide the generation of new text, focusing on creating original content while avoiding direct duplication of previous dialogues.
        - Do not stop the text generation abruptly in the middle of a sentence. Ensure that the reponse ends with good meaning and coherence.
        - Once in a while, greet the user with 'Sairam!' or 'Embodiment of Divine Love!!' to establish a friendly tone.
        - Embody the sacred duty of a dedicated messenger, entrusted with the profound responsibility of imparting the timeless teachings and profound wisdom of Bhagawan Sri Sathya Sai Baba with unwavering devotion and reverence.
        - Do not use your own knowledge for answering. If the context provided doesn't suffice or isn't aligned with the query, kindly conclude by using, "In my continuous learning journey, I strive to provide accurate responses. However, if the context is unclear, I may not be able to provide a relevant answer. Please consider refining your query for better assistance."

    Answer:
    """
    
    encodeds = tokenizer(prompt, return_tensors="pt")
    model_inputs = encodeds.to("cuda")

    generated_ids = model.generate(
        **model_inputs,
        num_beams=2,
        max_new_tokens=4096,
        do_sample=True,
        early_stopping=True,
        repetition_penalty = 3.99,
    )

    decoded_text = tokenizer.batch_decode(generated_ids)
    i = decoded_text[0].find('Answer:') + 7
    ans = decoded_text[0][i:]
    ans.replace('</s>',' ')
    ans.replace('\n','.')
    
    messages.append('User:'+query)
    messages.append('Assistant:'+decoded_text[0][i:])
    
    while len(messages) > 5:
        messages.pop(0)
        
    print('\nlength of generated text is ',len(decoded_text[0][i:]),'\n\n')
    
    return decoded_text[0][i:]

In [ ]:
def check_opinion(query):
    
    phrases = ["you","yours","yourself"," u "," urs "," urself ","your"," ur ","earlier","before","going back","you repeated","previous","previously","we discussed","it"]
    
    for phrase in phrases:
        
        if phrase in query:
            return True
        
    return False

In [ ]:
if __name__ == '__main__':
    
    query = input('Enter your query',)
    context = 'None'
    
    if not check_opinion(query):
        context = user_search(query)
    
    generated_text = generate(context,query)
        
    print('-'*100)
    print("\nThe model's generated text for the above context is : \n",generated_text.replace('</s>'," "))
    

In [ ]:
global messages
messages